In [1]:
# import comet_ml at the top of your file
from comet_ml import Experiment
experiment = Experiment(
            project_name="music-genre-multiclass-classification",
            workspace="wodenwang820118",
        )
import comet_ml
import logging

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/118810b8356b459b804bb73b0c0caca4



In [2]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger("comet_ml")

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras

%matplotlib inline
sns.set_style('whitegrid')

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [4]:
# tensorflow 2.7 
import tensorflow as tf 
from tensorflow.keras.layers import Input,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model, Sequential

In [5]:
# scale the numeric data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [6]:
df_train_features = pd.read_csv('data/train_features.csv')
df_train_labels = pd.read_csv('data/train_labels.csv')

df_valid_features = pd.read_csv('data/valid_features.csv')
df_valid_labels = pd.read_csv('data/valid_labels.csv')

df_test_features = pd.read_csv('data/test_features.csv')
df_test_labels = pd.read_csv('data/test_labels.csv')

In [7]:
num_train_data = df_train_features.iloc[:,9:]
num_valid_data = df_valid_features.iloc[:,9:]
num_test_data = df_test_features.iloc[:,9:]

In [8]:
num_train = num_train_data.astype('float64')
num_valid = num_valid_data.astype('float64')
num_test = num_test_data.astype('float64')

In [9]:
num_train = scaler.fit_transform(num_train_data)
num_valid = scaler.fit_transform(num_valid_data)
num_test = scaler.fit_transform(num_test_data)

In [10]:
num_train_y = pd.get_dummies(df_train_labels['genre'])
num_valid_y = pd.get_dummies(df_valid_labels['genre'])

In [11]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(df_train_labels['genre']),y=df_train_labels['genre'])
class_weight = {i :class_weight[i] for i in range(8)}

In [12]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=5)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=2, verbose=1, mode='auto', min_delta=0.00001, cooldown=0, min_lr=0.0001)

In [13]:
class AudioGenreClassifier:
    def __init__(self,num_train,num_train_y,num_valid,num_valid_y,class_weight,early_stop,experiment):
        self.num_train = num_train
        self.num_train_y = num_train_y
        self.num_valid = num_valid
        self.num_valid_y = num_valid_y
        self.class_weight = class_weight
        self.early_stop = early_stop
        self.experiment = experiment
    
    def build_model(self):
        # Build the model
        model = Sequential()
        model.add(
            Dense(
                self.experiment.get_parameter("first_layer_units"),
                activation='elu',
                input_shape=(num_train.shape[1],)
            )
        )
        model.add(Dropout(self.experiment.get_parameter("first_layer_dropout_rate")))
        model.add(BatchNormalization())

        model.add(
            Dense(
                self.experiment.get_parameter("second_layer_units"),
                activation='elu'))
        model.add(Dropout(self.experiment.get_parameter("second_layer_dropout_rate")))
        model.add(BatchNormalization())
        
        model.add(Dense(8,activation='softmax'))
        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model
    
    def train_model(self):
        # Train the model
        model = self.build_model()
        model.fit(
            self.num_train,
            self.num_train_y,
            batch_size=self.experiment.get_parameter("batch_size"),
            epochs=self.experiment.get_parameter("epochs"),
            validation_data=(self.num_valid,self.num_valid_y),
            class_weight=self.class_weight,
            callbacks=[self.early_stop, reduce_lr]
        )
        return model
    
    def evaluate_model(self):
        # Evaluate the model
        model = self.train_model()
        score = model.evaluate(self.num_valid,self.num_valid_y)
        LOGGER.info(f"{ score }")
    
    def grid_search(self, config_dict):
        opt = comet_ml.Optimizer(config_dict)
        for self.experiment in opt.get_experiments(project_name="music-genre-multiclass-classification"):
            self.experiment.log_parameters("epochs", 10)

            self.build_model()
            self.train_model()
            self.evaluate_model()
            self.experiment.end()

In [14]:
audio_model = AudioGenreClassifier(num_train,num_train_y,num_valid,num_valid_y,class_weight,early_stop,experiment)
# mu is the mean number of units, sigma is the standard deviation
audio_model.grid_search({
    "algorithm": "bayes",
    "name": "Optimize Music Classification Network",
    "spec": {"maxCombo": 10, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "first_layer_units": {"type": "discrete", "values": [128,256,300,400,450,500,550,600,700,800,900,1000]},
        "first_layer_dropout_rate": {"type": "discrete", "values": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]},
        "second_layer_units": {"type": "discrete", "values": [128,256,300,400,450,500,550]},
        "second_layer_dropout_rate": {"type": "discrete", "values": [0,0.1,0.2,0.3,0.4,0.5,0.6]},
        "batch_size": {"type": "discrete", "values": [16,32,64]},
        "epochs": {"type": "discrete", "values": [20,30,40]},
    },
    "trials": 1,
})

COMET INFO: COMET_OPTIMIZER_ID=91ab5f3a683b40e1a1ba1a615248e346
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 47628, 'endTime': None, 'id': '91ab5f3a683b40e1a1ba1a615248e346', 'lastUpdateTime': None, 'maxCombo': 10, 'name': 'Optimize Music Classification Network', 'parameters': {'batch_size': {'type': 'discrete', 'values': [16, 32, 64]}, 'epochs': {'type': 'discrete', 'values': [20, 30, 40]}, 'first_layer_dropout_rate': {'type': 'discrete', 'values': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}, 'first_layer_units': {'type': 'discrete', 'values': [128, 256, 300, 400, 450, 500, 550, 600, 700, 800, 900, 1000]}, 'second_layer_dropout_rate': {'type': 'discrete', 'values': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]}, 'second_layer_units': {'type': 'discrete', 'values': [128, 256, 300, 400, 450, 500, 550]}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 10, 'metric': 'loss', 'minSampleSize': 100, 'objective': 'minimize', 'retryAssignLimit': 0, 'retryLimit':

Epoch 1/20


COMET INFO: ignoring tensorflow summary log of metrics because of keras; set `comet_ml.loggers.tensorboard_logger.LOG_METRICS = True` to override


120/120 [==============================] - 2s 5ms/step - loss: 1.7524 - accuracy: 0.4048 - val_loss: 1.5924 - val_accuracy: 0.4000 - lr: 0.0010
Epoch 2/20
120/120 [==============================] - 0s 4ms/step - loss: 1.4387 - accuracy: 0.4751 - val_loss: 1.3349 - val_accuracy: 0.5133 - lr: 0.0010
Epoch 3/20
120/120 [==============================] - 0s 4ms/step - loss: 1.3597 - accuracy: 0.5018 - val_loss: 1.3207 - val_accuracy: 0.5044 - lr: 0.0010
Epoch 4/20
120/120 [==============================] - 1s 4ms/step - loss: 1.2939 - accuracy: 0.5130 - val_loss: 1.3260 - val_accuracy: 0.5356 - lr: 0.0010
Epoch 5/20
107/120 [=========================>....] - ETA: 0s - loss: 1.2395 - accuracy: 0.5273
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0007000000332482159.
120/120 [==============================] - 0s 4ms/step - loss: 1.2282 - accuracy: 0.5298 - val_loss: 1.4245 - val_accuracy: 0.4756 - lr: 0.0010
Epoch 6/20
120/120 [==============================] - 0s 3ms/step - los

COMET INFO: [1.32178795337677, 0.551111102104187]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/7b4471fdb3a54217baf21a1709782aa5
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [22]                : (0.4047929048538208, 0.6213857531547546)
COMET INFO:     batch_accuracy [264]         : (0.078125, 0.6875)
COMET INFO:     batch_loss [264]             : (0.8283485174179077, 3.6091320514678955)
COMET INFO:     epoch_duration [22]          : (0.3900000000003274, 1.844000000000051)
COMET INFO:     loss [22]                    : (0.9549474120140076, 1.7616511583328247)
COMET INFO:     lr [22]                      : (0.00024009999469853938, 0.0010000000474974513)
COMET INFO:     val_accuracy [22]            : (0.4000000059604645

...

COMET INFO: [1.317121148109436, 0.5288888812065125]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/8a3da9b736c440bbadb6f5ef86bf4f92
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [20]                : (0.3813492953777313, 0.5675957202911377)
COMET INFO:     batch_accuracy [960]         : (0.17045454680919647, 0.75)
COMET INFO:     batch_loss [960]             : (0.7580433487892151, 3.242428779602051)
COMET INFO:     epoch_duration [20]          : (1.4070000000001528, 3.092999999999847)
COMET INFO:     loss [20]                    : (1.1180001497268677, 1.7328376770019531)
COMET INFO:     lr [20]                      : (0.0003429999924264848, 0.0010000000474974513)
COMET INFO:     val_accuracy [20]            : (0.4600000

Epoch 1/20
120/120 [==============================] - 3s 17ms/step - loss: 1.7548 - accuracy: 0.4148 - val_loss: 1.4657 - val_accuracy: 0.4467 - lr: 0.0010
Epoch 2/20
120/120 [==============================] - 1s 9ms/step - loss: 1.4002 - accuracy: 0.4876 - val_loss: 1.3040 - val_accuracy: 0.5756 - lr: 0.0010
Epoch 3/20
120/120 [==============================] - 1s 9ms/step - loss: 1.3059 - accuracy: 0.5244 - val_loss: 1.3500 - val_accuracy: 0.5311 - lr: 0.0010
Epoch 4/20
117/120 [============================>.] - ETA: 0s - loss: 1.2376 - accuracy: 0.5385
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0007000000332482159.
120/120 [==============================] - 1s 8ms/step - loss: 1.2368 - accuracy: 0.5382 - val_loss: 1.3263 - val_accuracy: 0.5356 - lr: 0.0010
Epoch 5/20
120/120 [==============================] - 1s 9ms/step - loss: 1.1505 - accuracy: 0.5643 - val_loss: 1.3389 - val_accuracy: 0.5178 - lr: 7.0000e-04
Epoch 6/20
115/120 [===========================>..] - E

COMET INFO: [1.387366533279419, 0.5288888812065125]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/52fc529688274485ab1291c6297ae525
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [16]                : (0.4148215651512146, 0.6224277019500732)
COMET INFO:     batch_accuracy [192]         : (0.03125, 0.671875)
COMET INFO:     batch_loss [192]             : (0.8403425216674805, 3.841454029083252)
COMET INFO:     epoch_duration [16]          : (1.0, 2.813000000000102)
COMET INFO:     loss [16]                    : (0.982441782951355, 1.754837989807129)
COMET INFO:     lr [16]                      : (0.0004900000058114529, 0.0010000000474974513)
COMET INFO:     val_accuracy [16]            : (0.4466666579246521, 0.579999983310

Epoch 1/40
120/120 [==============================] - 2s 12ms/step - loss: 1.8803 - accuracy: 0.3695 - val_loss: 1.4225 - val_accuracy: 0.5133 - lr: 0.0010
Epoch 2/40
120/120 [==============================] - 1s 9ms/step - loss: 1.5847 - accuracy: 0.4370 - val_loss: 1.3865 - val_accuracy: 0.5289 - lr: 0.0010
Epoch 3/40
120/120 [==============================] - 1s 9ms/step - loss: 1.4813 - accuracy: 0.4535 - val_loss: 1.2942 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 4/40
120/120 [==============================] - 1s 9ms/step - loss: 1.4093 - accuracy: 0.4767 - val_loss: 1.3968 - val_accuracy: 0.5067 - lr: 0.0010
Epoch 5/40
115/120 [===========================>..] - ETA: 0s - loss: 1.3670 - accuracy: 0.4826
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0007000000332482159.
120/120 [==============================] - 1s 9ms/step - loss: 1.3699 - accuracy: 0.4820 - val_loss: 1.3388 - val_accuracy: 0.5222 - lr: 0.0010
Epoch 6/40
120/120 [==============================] - 1s 9m

COMET INFO: [1.374954104423523, 0.5177778005599976]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/7694cd6afe4d45ada141407fd17c437c
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [15]                : (0.36949726939201355, 0.5174524784088135)
COMET INFO:     batch_accuracy [180]         : (0.078125, 0.5625)
COMET INFO:     batch_loss [180]             : (0.9657741189002991, 2.794071912765503)
COMET INFO:     epoch_duration [15]          : (0.5780000000004293, 2.219000000000051)
COMET INFO:     loss [15]                    : (1.256218433380127, 1.8856014013290405)
COMET INFO:     lr [15]                      : (0.0004900000058114529, 0.0010000000474974513)
COMET INFO:     val_accuracy [15]            : (0.4288888871669769

In [19]:
model = Sequential()
model.add(
    Dense(
        900,
        activation='elu',
        input_shape=(num_train.shape[1],)
    )
)
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(Dense(550,activation='elu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(8,activation='softmax'))
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [20]:
model.fit(
    num_train,
    num_train_y,
    batch_size=64,
    epochs=40,
    validation_data=(num_valid,num_valid_y),
    class_weight=class_weight,
    callbacks=[early_stop, reduce_lr],
)

Epoch 1/40
120/120 [==============================] - 1s 6ms/step - loss: 1.7210 - accuracy: 0.4237 - val_loss: 1.4420 - val_accuracy: 0.4667 - lr: 0.0010
Epoch 2/40
120/120 [==============================] - 0s 4ms/step - loss: 1.3709 - accuracy: 0.5009 - val_loss: 1.5220 - val_accuracy: 0.4511 - lr: 0.0010
Epoch 3/40
108/120 [==========================>...] - ETA: 0s - loss: 1.2924 - accuracy: 0.5195
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0007000000332482159.
120/120 [==============================] - 0s 4ms/step - loss: 1.2880 - accuracy: 0.5220 - val_loss: 1.4514 - val_accuracy: 0.4978 - lr: 0.0010
Epoch 4/40
120/120 [==============================] - 0s 4ms/step - loss: 1.1625 - accuracy: 0.5624 - val_loss: 1.3469 - val_accuracy: 0.5644 - lr: 7.0000e-04
Epoch 5/40
120/120 [==============================] - 1s 4ms/step - loss: 1.1026 - accuracy: 0.5796 - val_loss: 1.2720 - val_accuracy: 0.5711 - lr: 7.0000e-04
Epoch 6/40
120/120 [==============================] 

In [21]:
from sklearn.metrics import classification_report
validation = model.predict(num_valid)
validation = validation.argmax(axis=1)
classes_mapping = {
    0: 'classic pop and rock',
    1: 'dance and electronica',
    2: 'folk',
    3: 'jazz and blues',
    4: 'metal',
    5: 'pop',
    6: 'punk',
    7: 'soul and reggae',
}
predict_label_array = np.vectorize(classes_mapping.get)(validation)
correct_labels = df_valid_labels['genre'].values
print(classification_report(correct_labels,predict_label_array))

                       precision    recall  f1-score   support

 classic pop and rock       0.41      0.51      0.46        55
dance and electronica       0.36      0.42      0.39        45
                 folk       0.44      0.58      0.50        64
       jazz and blues       0.44      0.39      0.41        44
                metal       0.87      0.73      0.79        66
                  pop       0.65      0.49      0.56        74
                 punk       0.42      0.50      0.46        44
      soul and reggae       0.73      0.55      0.63        58

             accuracy                           0.53       450
            macro avg       0.54      0.52      0.52       450
         weighted avg       0.56      0.53      0.54       450



In [22]:
from sklearn.metrics import classification_report
validation = model.predict(num_test)
validation = validation.argmax(axis=1)
classes_mapping = {
    0: 'classic pop and rock',
    1: 'dance and electronica',
    2: 'folk',
    3: 'jazz and blues',
    4: 'metal',
    5: 'pop',
    6: 'punk',
    7: 'soul and reggae',
}
predict_label_array = np.vectorize(classes_mapping.get)(validation)
correct_labels = df_test_labels['genre'].values
print(classification_report(correct_labels,predict_label_array))

                       precision    recall  f1-score   support

 classic pop and rock       0.26      0.41      0.32        64
dance and electronica       0.67      0.52      0.58        64
                 folk       0.51      0.51      0.51        73
       jazz and blues       0.08      0.06      0.07        36
                metal       0.60      0.50      0.55        42
                  pop       0.37      0.50      0.42        44
                 punk       0.25      0.30      0.27        40
      soul and reggae       0.61      0.35      0.45        65

             accuracy                           0.41       428
            macro avg       0.42      0.39      0.40       428
         weighted avg       0.44      0.41      0.42       428

